In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import yaml
import dask.array as da
from ag3 import release_data
v3_release = release_data()

### get meta

In [55]:
#samplesets
samplesets = v3_release.all_wild_sample_sets
df_meta = v3_release.load_sample_set_metadata(samplesets, True)
#fixes
df_meta.country = df_meta.country.str.replace("Gambia, The", "The Gambia").str.replace("Equatorial Guinea", "Bioko")
#consolidate all Mayotte sites into one (so we get a pie chart)
df_meta.loc[(df_meta.country == 'Mayotte'),'latitude'] = df_meta[df_meta.country == 'Mayotte'].latitude.mean()
df_meta.loc[(df_meta.country == 'Mayotte'),'longitude'] = df_meta[df_meta.country == 'Mayotte'].longitude.mean()
#counts by sites
df_site_counts = df_meta.groupby(['latitude', 'longitude', 'country']).size().reset_index(name='counts')
df_site_counts['site'] = range(len(df_site_counts))
#>= 10 sites 
df_10 = df_site_counts[df_site_counts.counts >= 10].copy()
len(df_10),len(df_10.latitude.unique()),len(df_10.longitude.unique())

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


(43, 43, 42)

In [57]:
#new full meta but only containing >=10 sites (to preserve all species columns)
df_10_meta = df_meta.merge(df_10[['longitude','latitude', 'site']])
len(df_10_meta.latitude.unique()),len(df_10_meta.longitude.unique())

(43, 42)

### get crosstabbing

In [35]:
arab_call = pd.crosstab(df_10_meta.site, df_10_meta.species_gambcolu_arabiensis)
colu_call = pd.crosstab(df_10_meta.site, df_10_meta.species_gambiae_coluzzii)
colu_call.rename({"intermediate": "gambiae/coluzzii"}, axis=1, inplace=True)
colu_call.head()

species_gambiae_coluzzii,coluzzii,gambiae,gambiae/coluzzii
site,,,
0,0,74,0
2,0,23,0
3,81,0,0
4,0,36,6
5,0,28,45


In [5]:
sex_call = pd.crosstab(df_10_meta.site, df_10_meta.sex_call)
sex_call.rename({"F": "female", "M": "male", "UKN": "sex_unknown"}, axis=1, inplace=True)
sex_call.head()

sex_call,female,male,sex_unknown
site,,,
0,74,0,0
1,41,0,0
2,11,12,0
3,77,4,0
4,43,0,0


### get building

- fix types
- rename and re-order columns
- need to discuss the location - some sites have multiple locations for same lon/lat

In [24]:
#reindex and join
df_10 =  df_10.set_index('site')
df_10_final = df_10.join([arab_call, colu_call, sex_call])

In [70]:
#add locations - must be a better way of doing this using merge maybe?
site = []
for _, row in df_10_final.iterrows():
    all_site = df_meta[(df_meta.latitude == row.latitude) & (df_meta.longitude == row.longitude)].copy()
    if all_site.location.unique()[0] != 'Mtsamboro Forest Reserve':
        site.append(all_site.location.unique()[0])
    else:
        site.append('Mayotte')
df_10_final['site'] = site
#fix_mayotte

In [74]:
df_10_final.head()

,latitude,longitude,country,counts,site,arabiensis,gamb_colu,intermediate,coluzzii,gambiae,gambiae/coluzzii,female,male,sex_unknown
0,-23.716000,35.299000,Mozambique,74,Furvela,0,74,0,0.0,74.0,0.0,74,0,0
1,-15.933000,34.755000,Malawi,41,Chikhwawa,41,0,0,NaN,NaN,NaN,41,0,0
2,-12.857087,45.137435,Mayotte,23,Mayotte,0,23,0,0.0,23.0,0.0,11,12,0
3,-8.884000,13.302000,Angola,81,Luanda,0,81,0,81.0,0.0,0.0,77,4,0
4,-4.940000,38.948000,Tanzania,43,Muheza,1,42,0,0.0,36.0,6.0,43,0,0


### get paid